In [1]:
# Import necessary packages
import stim
import numpy as np
import copy
from typing import List, Tuple

In [69]:
class PauliSummand:
    def __init__(self, phase: complex, pauli: stim.PauliString):
        """ 
        Representation for single element of 'PauliSum'

        Params:
        -------
        phase - Phase of the 'PauliSummand' instance
        pauli - Pauli operator of the 'PauliSummand' instance
        """
        self.phase = phase 
        self.pauli = pauli 
        
        # Store basis paulis for each 'PauliSummand' instance
        self.bases = []

    def get_phase(self):
        """ 
        Get phase of PauliSummand

        Returns:
        --------
        self.phase
        """
        return self.phase 

    def set_phase(self, phase: complex):
        """ 
        Set phase of PauliSummand 

        Params:
        -------
        phase - Phase to set to of 'PauliSummand' instance
        """
        self.phase = phase

    def get_pauli(self):
        """ 
        Get pauli of PauliSummand

        Returns:
        --------
        self.pauli
        """
        return self.pauli

    def set_pauli(self, pauli: stim.PauliString):
        """ 
        Set pauli of PauliSummand 

        Params:
        -------
        pauli - Pauli to set to of 'PauliSummand' instance
        """
        self.pauli = pauli

    def get_bases(self):
        """ 
        Get list of basis elements that generate 'PauliSummand' instance

        Returns:
        --------
        self.bases
        """
        return self.bases 

    def add_to_basis(self, basis_ind: int):
        """ 
        Add basis index corresponding to index of basis element in overall list of 
        Pauli bases

        Params:
        -------
        basis_ind - Index of basis element in overall list of Pauli bases
        """
        self.bases.append(basis_ind)
        
    def get_pauli_str(self):
        """ 
        Extract just pauli string from stim Pauli representation
        """
        pauli_str = ""
        num_qubits = len(self.pauli)
        for i in range(num_qubits - 1, -1, -1):
            pauli_str = str(self.pauli)[i] + pauli_str
        return pauli_str

    def __str__(self):
        pauli_str = ""
        pauli_plus_phase = str(self.pauli)
        pauli_plus_phase_len = len(pauli_plus_phase)
        count = pauli_plus_phase_len - 1
        while (pauli_plus_phase[count] == '_' or pauli_plus_phase[count] == 'X' or 
            pauli_plus_phase[count] == 'Y' or pauli_plus_phase[count] == 'Z'):
            pauli_str = pauli_plus_phase[count] + pauli_str 
            count -= 1
        return "[" + str(self.phase) + " " + pauli_str + "]"

class PauliSum: 
    def __init__(self, *argv):
        """ 
        Representation of contraction of QCSAT instance
        """
        if len(argv) == 0:
            self.sum = []
            self.bases = []
            self.num_bases = 0
            self.summand_to_bases = {}
        else:
            self.sum = [argv[0]]
            self.bases = [argv[0]]
            self.num_bases = 1
            self.summand_to_bases = {str(argv[0].get_pauli()):[0]}

    def get_sum(self):
        """ 
        Get PauliSum in list representation
        """
        return self.sum 

    def get_bases(self):
        """ 
        Get bases for PauliSum
        """
        return self.bases 

    def get_basis_elem(self, pos: int):
        """
        Get basis element
        """ 
        return self.bases[pos]

    def set_basis_elem(self, pos: int, basis_elem: stim.PauliString):
        """ 
        Set a particular basis element
        """
        self.bases[pos] = basis_elem

    def set_bases(self, basis_list: List):
        """ 
        Set basis for PauliSum

        Params:
        -------
        basis_list - List of bases for PauliSum
        """
        self.bases = basis_list 

    def get_num_bases(self):
        """ 
        Get number of bases in PauliSum
        """
        return self.num_bases

    def set_num_bases(self, basis_count: int):
        """ 
        Set number of basis elements in PauliSum
        """
        self.num_bases = basis_count

    def add_to_bases(self, basis_elem: stim.PauliString):
        """
        Add a basis element to self.bases
        """ 
        self.bases.append(basis_elem)
        self.incr_num_bases()

    def incr_num_bases(self):
        """ 
        Increment 'num_bases' variable
        """
        self.num_bases += 1

    def __str__(self):
        str_out = ""
        for summand in self.sum:
            str_out += str(summand)
            str_out += " + "
        return str_out[:len(str_out) - 3]
        
    def combine_like_terms(self):
        """ 
        Combine like terms of Pauli Sum
        """
        pass 

    def add_sum_to_sum(self, p):
        """ 
        Add a PauliSum instance to 'self'

        Params:
        -------
        p - PauliSum instance to be added
        """
        self.sum += p.sum 

    def add_summand(self, s: PauliSummand):
        """ 
        Add a PauliSummand to 'self'

        Params:
        -------
        s - PauliSummand instance to be added 
        """
        self.sum.append(s)

    def get_summand_bases_dict(self):
        """ 
        Get 'summand_to_bases' dict
        """
        return self.summand_to_bases

    def retrieve_gen_basis(self, dict_key: str):
        """ 
        Returns generating set of Paulis for given PauliSummand
        """
        return self.summand_to_bases[dict_key] 

    def append_gen_basis(self, dict_key: str, gen_basis: int):
        """ 
        Appends a generating basis element to list associated with particular PauliSummand
        """
        if (gen_basis not in self.summand_to_bases[dict_key]):
            self.summand_to_bases[dict_key].append(gen_basis)

    def update_summand_bases_dict(self, gen_basis: int, basis_to_add: int):
        """ 
        Update 'summand_to_bases' in order to reflect transformation of basis
        """
        for key in self.summand_to_bases:
            if (gen_basis in self.retrieve_gen_basis(key)):
                self.append_gen_basis(key, basis_to_add)

    def add_to_summand_bases_dict(self, summand: str, basis_num: int):
        """ 
        Add a PauliSummand to 'summand_to_bases' dict 
        """
        if (summand in self.summand_to_bases):
            self.summand_to_bases[summand].append(basis_num)
        else:
            self.summand_to_bases[summand] = [basis_num]

    def transform_basis(self, t_gate_loc: int):
        """ 
        Transform basis so that T-gate can be appropriately applied to basis paulis. 
        We want only two bases acting non-trivially at the position where the T-gate 
        is to be applied.

        Params:
        -------
        t_gate_loc - Qubit location where T-gate is being applied
        """
        pauli_dict = {0: '_', 1: 'X', 2: 'Y', 3: 'Z'}
        pauli_column = [pauli_dict[p.__getitem__(t_gate_loc)] for p in self.bases]
        new_basis_paulis = []
        x_count = 0
        x_pos = 0
        y_count = 0
        y_pos = 0
        z_count = 0
        z_pos = 0
        for i in range(len(pauli_column)):
            if (pauli_column[i] == '_'):
                new_basis_paulis.append(self.get_basis_elem(i))
            if (pauli_column[i] == 'X'):
                if (x_count == 0):
                    new_basis_paulis.append(self.get_basis_elem(i))
                    x_pos = i
                else:
                    new_basis_paulis.append(self.get_basis_elem(x_pos) * self.get_basis_elem(i))
                    self.update_summand_bases_dict(i, x_pos)
                x_count += 1

            if (pauli_column[i] == 'Y'):
                if (y_count == 0):
                    new_basis_paulis.append(self.get_basis_elem(i))
                    y_pos = i
                else:
                    new_basis_paulis.append(self.get_basis_elem(y_pos) * self.get_basis_elem(i))
                    self.update_summand_bases_dict(i, y_pos)
                y_count += 1

            if (pauli_column[i] == 'Z'):
                if (z_count == 0):
                    new_basis_paulis.append(self.get_basis_elem(i))
                    z_pos = i
                else:
                    new_basis_paulis.append(self.get_basis_elem(z_pos) * self.get_basis_elem(i))
                    self.update_summand_bases_dict(i, z_pos)
                z_count += 1 

        if (x_count >= 1 and y_count >= 1):
            new_basis_paulis[x_pos] = new_basis_paulis[x_pos] * new_basis_paulis[y_pos]
            self.update_summand_bases_dict(x_pos, y_pos)

        self.set_bases(new_basis_paulis)

    def apply_t_gate(self, num_qubits: int, t_gate_loc: int = 0):
        """ 
        Simulate application of T gate on PauliSum based on which qubit T gate is 
        being applied to 

        Params:
        -------
        num_qubits - Number of qubits
        t_gate_loc - Location where T gate is being applied
        """
        phase_term = 1/np.sqrt(2)
        pauli_dict = {0: '_', 1: 'X', 2: 'Y', 3: 'Z'}
        added = False
        is_empty = True
        # new_sum = PauliSum()
        X_term = stim.PauliString('_' * t_gate_loc + 'X' + '_' * (num_qubits - t_gate_loc - 1))
        Y_term = stim.PauliString('_' * t_gate_loc + 'Y' + '_' * (num_qubits - t_gate_loc - 1))

        # Update Pauli Basis
        self.transform_basis(t_gate_loc)
        pauli_column = [pauli_dict[p.__getitem__(t_gate_loc)] for p in self.bases]
        for i in range(len(pauli_column)):
            if (pauli_column[i] == '_' or pauli_column[i] == 'Z'):
                continue 
            elif (pauli_column[i] == 'X'):
                new_basis = self.get_basis_elem(i) * X_term * Y_term
                self.add_to_bases(new_basis)
                #self.add_to_summand_bases_dict(str(new_basis), self.get_num_bases()) 
            else:
                new_basis = self.get_basis_elem(i) * Y_term * X_term 
                self.add_to_bases(new_basis)
                #self.add_to_summand_bases_dict(str(new_basis), self.get_num_bases())
                self.set_basis_elem(i, self.get_basis_elem(i) * -1)
        
        # Update Pauli Summands
        for summand in self.sum:
            #print("Pauli and t_gate_loc: " + str(summand.get_pauli().__getitem__(t_gate_loc)))
            if (pauli_dict[summand.get_pauli().__getitem__(t_gate_loc)] == '_' or pauli_dict[summand.get_pauli().__getitem__(t_gate_loc)] == 'Z'):
                #print("Identity or Z")
                continue
            elif (pauli_dict[summand.get_pauli().__getitem__(t_gate_loc)] == 'X'):
                #print("X")
                new_summand = PauliSummand(phase_term * summand.get_phase(), summand.get_pauli() * X_term * Y_term)
                summand.set_phase(summand.get_phase() * phase_term)
                if (added == False):
                    new_sum = PauliSum(new_summand)
                    is_empty = False
                    added = True 
                else:
                    new_sum.add_summand(new_summand)
                #self.add_to_summand_bases_dict(str(new_summand.get_pauli())) 
            else:
                #print("Y")
                new_summand = PauliSummand(phase_term * summand.get_phase(), summand.get_pauli() * Y_term * X_term)
                summand.set_phase(-1 * summand.get_phase() * phase_term)
                if (added == False):
                    new_sum = PauliSum(new_summand)
                    is_empty = False
                    added = True 
                else:
                    new_sum.add_summand(new_summand)

        if (not is_empty):
            self.add_sum_to_sum(new_sum)

    def generate_succinct_basis_term(self, k: int, num_qubits: int) -> stim.PauliString:
        """ 
        Generate new basis element A^(k)

        Params:
        -------
        k - Index related to the k-th position in Pauli String
        num_qubits - Number of qubits
        """
        gamma = lambda i,j: self.get_basis_elem(i).commutes(self.get_basis_elem(j))
        for i in range(k):
            if (gamma(i,k) == False):
                pauli_str = '_' * i + 'X' + '_' * (k - i - 1) + 'Z' + '_' * (num_qubits - k - 1)
            else:
                pauli_str = '_' * k + 'Z' + '_' * (num_qubits - k - 1)
        new_basis_elem = stim.PauliString(pauli_str)

        return new_basis_elem 

    def succinct_basis(self, num_qubits: int):
        """
        Derive a more succinct basis as prescribed by description of Lemma 17

        Params:
        -------
        num_qubits - Number of qubits
        """ 
        new_basis = [stim.PauliString("Z" + "_" * (num_qubits - 1))]
        for i in range(1, self.num_bases):
            new_basis.append(self.generate_succinct_basis_term(i, num_qubits))
        self.set_bases(new_basis)
    
    def apply_tableau(self, t: stim.Tableau):
        """ 
        Apply current stabilizer tableau on PauliSum instance

        Params:
        -------
        t - stabilizer tableau
        """
        # Apply tableau to PauliSum
        for summand in self.sum:
            new_summand = t(summand.get_pauli())
            summand.set_phase(summand.get_phase() * new_summand.sign)
            summand.set_pauli(new_summand)

        # Apply tableau to PauliSum basis
        self.set_bases([t(p) for p in self.get_bases()])

        # Update keys of dictionary
        for key in list(self.summand_to_bases):
            new_key = str(t(stim.PauliString(key)))
            self.summand_to_bases[new_key] = self.summand_to_bases.pop(key)
            

In [70]:
def sim_circ(num_qubits: int, circ_depth: int, num_t_gate: int) -> PauliSum:
    """
    Calculates U^{\dag}ZU for arbitrary Clifford + T circuit with a 
    single T gate acting WLOG on the first qubit

    Parameters:
    -----------
    * num_tot_qubits - Number of total qubits in system
    * circ_depth - Number of layers of operators in our circuit
    * num_t_gate - Number of t-gates to be inserted in our circuit

    Returns:
    ---------
    pauli_sum - Instance of PauliSum class
    """

    t = stim.Tableau(num_qubits)
    gates_applied = []
    basis_paulis = []
    t_gate_layer_loc = [] # List of circuit layer positions corresponding to T-gate application
    t_gate_qubit_loc = [] # Which qubit to apply the t-gate to.
    t_gate_count = 0;
    curr_layer = 0

    pauli_z_string = "Z" + "_" * (num_qubits - 1)
    z = stim.PauliString(pauli_z_string)
    z_phase = 1 + 0j
    z_summand = PauliSummand(z_phase, z)
    pauli_sum = PauliSum(z_summand)
    basis_paulis.append(z)
    pauli_sum.set_bases(basis_paulis)

    # Create list of circuit depth positions where we would like to apply T-gate
    for _ in range(num_t_gate):
        num_to_add = np.random.randint(0, circ_depth)
        if (num_to_add in t_gate_layer_loc):
            continue
        else:
            t_gate_layer_loc.append(num_to_add)
    
    # Create list of qubit positions where we would like to apply T-gate in each layer
    for _ in range(circ_depth):
        num_to_add = np.random.randint(0, num_qubits)
        t_gate_qubit_loc.append(num_to_add)

    # Need to sort this array so that application of T-gates makes chronological sense with elements of 't_gate_layer_loc'
    t_gate_layer_loc = np.sort(t_gate_layer_loc)
    print("Qubit location of T-gates: " + str(t_gate_qubit_loc))
    print("Layer location of T-gates: " + str(t_gate_layer_loc))
    for i in range(circ_depth):
        if (t_gate_count < len(t_gate_layer_loc) and i == t_gate_layer_loc[t_gate_count]):
            pauli_sum.apply_t_gate(num_qubits, t_gate_qubit_loc[curr_layer])

            #print("Intermediate T gate: " + str(pauli_sum))

            t_gate_count += 1
            t = stim.Tableau(num_qubits)
            gates_applied.append('T' + ' ' + str(t_gate_qubit_loc[curr_layer]))
            
        else:
            gate_to_add = stim.Tableau.random(num_qubits)
            pauli_sum.apply_tableau(gate_to_add)
            #print("Intermediate Clifford gate: " + str(pauli_sum))
            gates_applied.append(gate_to_add)

        curr_layer += 1
        print("Layer " + str(i) + ":" + str(pauli_sum))
    #print(gates_applied)
    return pauli_sum

In [71]:
num_wit_qubits = 5 # variable 'n' in paper
# For now, what the witness qubits are initialized to be doesn't matter (I THINK)
# as we are trying to minimize Val, given some optimal input states.
wit_qubits = np.zeros(num_wit_qubits, dtype=complex)

num_anc_qubits = 0 # variable 'm' in paper
anc_qubits = np.zeros(num_anc_qubits, dtype=complex)

num_t_gates = 3 # variable 't' in paper

num_meas_qubits = 1 # variable 'k' in paper
total_qubits = num_wit_qubits + num_anc_qubits

num_layers = 10

assert(num_layers > 0)

In [85]:
p_sum = sim_circ(total_qubits, num_layers, num_t_gates)
print("Pauli Sum: " + str(p_sum))
print("Basis: " + str(p_sum.get_bases()))
print("Summand generating bases: " + str(p_sum.get_summand_bases_dict()))
print("Number of basis elements: " + str(p_sum.get_num_bases()))
p_sum.succinct_basis(total_qubits)
print("Succinct basis: " + str(p_sum.get_bases()))


Qubit location of T-gates: [3, 3, 0, 3, 4, 3, 3, 4, 2, 2]
Layer location of T-gates: [3 5 6]
Layer 0:[(-1+0j) YYYZ_]
Layer 1:[(-1+0j) _X_Z_]
Layer 2:[(1-0j) YZYYY]
Layer 3:[(-0.7071067811865475+0j) YZYYY] + [(0.7071067811865475+0j) YZYXY]
Layer 4:[(0.7071067811865475-0j) XXYXZ] + [(-0.7071067811865475+0j) Z__YY]
Layer 5:[(0.4999999999999999+0j) XXYXZ] + [(0.4999999999999999+0j) Z__YY] + [(0.4999999999999999+0j) XXYYZ] + [(-0.4999999999999999+0j) Z__XY]
Layer 6:[(0.3535533905932737+0j) XXYXZ] + [(-0.3535533905932737+0j) Z__YY] + [(-0.3535533905932737+0j) XXYYZ] + [(-0.3535533905932737+0j) Z__XY] + [(0.3535533905932737+0j) XXYYZ] + [(0.3535533905932737+0j) Z__XY] + [(0.3535533905932737+0j) XXYXZ] + [(-0.3535533905932737+0j) Z__YY]
Layer 7:[(0.3535533905932737+0j) XX_ZY] + [(0.3535533905932737-0j) XYXZY] + [(-0.3535533905932737+0j) XYX_Y] + [(0.3535533905932737-0j) XX__Y] + [(0.3535533905932737+0j) XYX_Y] + [(-0.3535533905932737+0j) XX__Y] + [(0.3535533905932737+0j) XX_ZY] + [(0.353553390